In [ ]:
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os, warnings, cv2
seed = 2022
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/세미(2) 프로젝트/2.데이터

/content/drive/.shortcut-targets-by-id/1iZJ97_6lEv6dD796nV-fw4WzH5bf62Jz/세미(2) 프로젝트/2.데이터


In [ ]:
# 파일 이름 읽어오기 (레이블로 사용 가능)
categories = os.listdir()[:8]
print(categories)

['isang', 'jammanbo', 'evee', 'pie', 'squirtle', 'pikachu', 'jiwoo', 'leeseul']


## 1.데이터 전처리
- resize
- morphologyEx  : 노이즈 제거

In [ ]:
# 전역 변수부 
categories = os.listdir()[:-1]
X = []
y = []
err = []

# 메인부
for idx, name in enumerate(tqdm(categories)):

    filepath = './'+ name + '/'
    # 읽어들일 파일 경로 설정
    for path in os.listdir(filepath):

        img = cv2.imread(filepath + path)
        try:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, dsize=(128, 128), interpolation= cv2.INTER_LINEAR)

            # 노이즈 제거 - GaussianBlur (5, 5) 사용
            
            #blur = cv2.GaussianBlur(img, ksize = (5, 5), sigmaX = 0)
            # 선 따기
            #edged = cv2.Canny(blur, 200, 255)

            X.append(img / 256) #.reshape(128, 128, 3)
            y.append(idx)

        except:
            err.append(filepath + path)
            continue

X = np.array(X)
y = to_categorical(np.array(y))     # 원핫인코딩 수행

print('\n에러:','None' if len(err) == 0 else err)

100%|██████████| 8/8 [00:53<00:00,  6.72s/it]



에러: ['./jiwoo/jiwoo (367).jpg']


In [ ]:
print(y[0])
print(type(y))

[1. 0. 0. 0. 0. 0. 0. 0.]
<class 'numpy.ndarray'>


In [ ]:
X.shape, type(X[0]) 

((3999, 128, 128, 3), numpy.ndarray)

In [ ]:
 # x.reshape(1, 24, 24, 3)

## 2.train/test split
- train 데이터로 train/valid split 진행
- train 데이터로 데이터 부풀리기 수행


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, stratify = y, random_state = seed
)

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(
    X_train, y_train, stratify = y_train, test_size = 0.2, random_state = seed
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,          # randomly rotate images in the range 15 degree
    width_shift_range=0.1,      # randomly shift images horizontally 
    height_shift_range=0.1,     # randomly shift images vertically
    shear_range=0.1,            # 도형의 기울기
    zoom_range=[0.8,1.2],       # 0.8~1.2 배 크기로 임의로 확대/축소
    horizontal_flip=True,       # randomly flip images
    vertical_flip=False,        # randomly flip images
    fill_mode='nearest'         # set mode for filling points outside the input boundaries
)

In [ ]:
datagen.fit(X_train)

In [ ]:
datagen.flow(X_train, y = None, batch_size = 1, save_to_dir = './데이터부풀리기/노이즈제거',
                   save_prefix = 'pre')

In [ ]:
for i, _ in enumerate(datagen.flow(X_train, batch_size = 1, save_to_dir = './데이터부풀리기/노이즈제거',
                                   save_prefix = 'pre',  save_format = 'jpg')):
    if i == :
        break

In [ ]:
X_train[0].shape, X_train[0].shape[1:]

((128, 128, 3), (128, 3))

In [ ]:
train_datagenerator = datagen.flow_from_directory(
    './데이터부풀리기/노이즈제거', target_size = (128, 128),
    batch_size = 3, class_mode = 'categorical'
)

Found 0 images belonging to 0 classes.


## 3.모델 정의 및 설정

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import RMSprop, Adam

---
- 데이터 부풀리기 없이 진행





In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape = X_train.shape[1:], padding = 'same', activation = 'relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(64, (3, 3), activation = 'relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation = 'softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 64, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 62, 62, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                    

In [ ]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

model_path = './모델저장/poketmon_adam.h5'
mc1 = ModelCheckpoint(model_path, moniter = 'val_loss', verbose = 1, save_best_only = True)
es1 = EarlyStopping(patience = 15)

In [ ]:
hist1 = model.fit(
    X_train, y_train, batch_size=32, epochs = 100, callbacks=[mc1, es1]
)

Epoch 1/100
100/100 [==============================] - 15s 35ms/step - loss: 1.4036 - accuracy: 0.5236
Epoch 2/100
100/100 [==============================] - 3s 30ms/step - loss: 0.6982 - accuracy: 0.7684
Epoch 3/100
100/100 [==============================] - 3s 30ms/step - loss: 0.5310 - accuracy: 0.8237
Epoch 4/100
100/100 [==============================] - 3s 30ms/step - loss: 0.4252 - accuracy: 0.8581
Epoch 5/100
100/100 [==============================] - 3s 30ms/step - loss: 0.3712 - accuracy: 0.8803
Epoch 6/100
100/100 [==============================] - 3s 30ms/step - loss: 0.2767 - accuracy: 0.9084
Epoch 7/100
100/100 [==============================] - 3s 30ms/step - loss: 0.2535 - accuracy: 0.9222
Epoch 8/100
100/100 [==============================] - 3s 30ms/step - loss: 0.2216 - accuracy: 0.9272
Epoch 9/100
100/100 [==============================] - 3s 30ms/step - loss: 0.1675 - accuracy: 0.9481
Epoch 10/100
100/100 [==============================] - 3s 32ms/step - loss: 0.14

In [ ]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

OSError: ignored